In [1]:
#------------------------------------------- bibliotecas

import requests # raspagem do conteúdo da página
from bs4 import BeautifulSoup as bs # estruturação do conteúdo raspado
import boto3 # conexão com a AWS

In [ ]:
#------------------------------------------- configurações AWS

s3 = boto3.client('s3') # instância para conectar à AWS

# configurações do bucket
bucket = 'elasticbeanstalk-us-east-2-047710767346'

# especificando e criando os prefixos (equivalentes a "subpastas")
prefixo_mae = 'Case_A3_refatorado'
s3.put_object(Bucket=bucket, Key=f'{prefixo_mae}/')

prefixo_raw = f'{prefixo_mae}/raw_arquivos_zip'
s3.put_object(Bucket=bucket, Key=f'{prefixo_raw}/')

In [24]:
#------------------------------------------- raspagem a página 1 - encontrando os links para download

url = 'https://www.gov.br/inep/pt-br/acesso-a-informacao/dados-abertos/microdados/censo-da-educacao-superior'

# etapa adicional, para impedir que a mensagem de erro apareça a cada requisição
from requests.packages.urllib3.exceptions import InsecureRequestWarning
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

html_bruto = requests.get(url, verify=False) # conteúdo bruto da página, o "sopão" de infos

# configurando o parser para organizar e ver os dados de maneira estruturara (para achar a div alvo)
html_estruturado = bs(html_bruto.content, 'html.parser')

# separando a div alvo, identificada na inspeção da página - "parent_div"
parent_div = html_estruturado.find('div', {'id': 'parent-fieldname-text'})

In [26]:
#------------------------------------------- raspagem a página 2 - separando e armazenando os links para download

# lista para armazenar os links para download, que vamos extrair da div
links_download = []

# percorrendo os elementos da div para encontrar os links
for year_element in parent_div.find_all('p', {'class': 'callout'}):
    ano = year_element.find('strong').text.strip()
    ul_element = year_element.find_next('ul')

    for li_element in ul_element.find_all('li'):
        link = li_element.find('a')['href']
        links_download.append((ano, link)) # armazenando o link na lista criada

In [37]:
#------------------------------------------- loop para download em massa dos arquivos zip

# agora, vamos iterar sobre os links listados para fazer o download e salvar os arquivos direto no bucket
for ano, link in links_download:
    try:
        # faz o download e desativa a verificação do certificado SSL
        arquivo_zip = requests.get(link, stream=True, verify=False)
        arquivo_zip.raise_for_status()  # verifica se houve erro no download

        # envia o arquivo para o bucket do S3 dentro da subpasta "arquivos_zip"
        nome_completo_zip = f'{prefixo_raw}/Microdados_{ano}.zip'
        s3.upload_fileobj(    # fileobj é o 'objeto do arquivo' a ser enviado para o bucket
            arquivo_zip.raw,  # raw é o conteúdo do arquivo e indica que o arquivo não foi aberto (camada 'raw', camada 'de abstração')
            bucket,           # nome do bucket
            nome_completo_zip # nome completo do arquivo no bucket >>> caminho(prefixos)+nome(microdados_ano.zip)
        )

        print(f'O arquivo zip com os dados de {ano} foi escrito com sucesso. Caminho: {nome_completo_zip}')

    except Exception as e:
        print(f'Erro ao baixar o arquivo Microdados_{ano}: {e}')

O arquivo zip com os dados de 2022 foi escrito com sucesso. Caminho: Case_A3_refatorado/raw_arquivos_zip/Microdados_2022.zip
O arquivo zip com os dados de 2021 foi escrito com sucesso. Caminho: Case_A3_refatorado/raw_arquivos_zip/Microdados_2021.zip
O arquivo zip com os dados de 2020 foi escrito com sucesso. Caminho: Case_A3_refatorado/raw_arquivos_zip/Microdados_2020.zip
O arquivo zip com os dados de 2019 foi escrito com sucesso. Caminho: Case_A3_refatorado/raw_arquivos_zip/Microdados_2019.zip
O arquivo zip com os dados de 2018 foi escrito com sucesso. Caminho: Case_A3_refatorado/raw_arquivos_zip/Microdados_2018.zip
O arquivo zip com os dados de 2017 foi escrito com sucesso. Caminho: Case_A3_refatorado/raw_arquivos_zip/Microdados_2017.zip
O arquivo zip com os dados de 2016 foi escrito com sucesso. Caminho: Case_A3_refatorado/raw_arquivos_zip/Microdados_2016.zip
O arquivo zip com os dados de 2015 foi escrito com sucesso. Caminho: Case_A3_refatorado/raw_arquivos_zip/Microdados_2015.zip
